In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px

/Users/catedombrowski/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
wine = pd.read_csv('wine.csv')
wine

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,white
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,white
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,white
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,white


In [3]:
#standardize the columns

columns_to_standardize = ['fixed.acidity', 'volatile.acidity', 'citric.acid', 'residual.sugar', 'chlorides',
                          'free.sulfur.dioxide', 'total.sulfur.dioxide', 'density', 'pH', 'sulphates',
                          'alcohol', 'quality']

scaler = StandardScaler()

# fit and transform the data
wine[columns_to_standardize] = scaler.fit_transform(wine[columns_to_standardize])

In [4]:
#specify hyperparameters

tsne = TSNE(n_components=2, verbose=1, perplexity=30, n_iter=1000, learning_rate=200)

In [5]:
# run t-SNE on the standardized data
tsne_results = tsne.fit_transform(wine[columns_to_standardize])

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 6497 samples in 0.021s...
[t-SNE] Computed neighbors for 6497 samples in 0.856s...
[t-SNE] Computed conditional probabilities for sample 1000 / 6497
[t-SNE] Computed conditional probabilities for sample 2000 / 6497
[t-SNE] Computed conditional probabilities for sample 3000 / 6497
[t-SNE] Computed conditional probabilities for sample 4000 / 6497
[t-SNE] Computed conditional probabilities for sample 5000 / 6497
[t-SNE] Computed conditional probabilities for sample 6000 / 6497
[t-SNE] Computed conditional probabilities for sample 6497 / 6497
[t-SNE] Mean sigma: 0.781486
[t-SNE] KL divergence after 250 iterations with early exaggeration: 82.350372
[t-SNE] KL divergence after 1000 iterations: 1.422137


In [10]:
# convert the t-SNE results to a DataFrame
tsne_df = pd.DataFrame(data = tsne_results, columns = ['Dim1', 'Dim2'])

# add the color to this DataFrame
tsne_df['color'] = wine['color'].values

# define 'red' or 'wine' category
wine.loc[wine['color']== 'red'.index, 'color_category'] = 'Smallest'
wine.loc[wine['color']== 'white'.index, 'color_category'] = 'Largest'


# create an interactive plot
fig = px.scatter(tsne_df, x='Dim1', y='Dim2', color='color', hover_data=['color'])
fig.show()

#### The tSNE does a much better job at clustering between red and white wines than the K means cluster. 

In [7]:
# define 'High Quality', 'Low Quality' or 'Other' category
wine['quality_category'] = 'Other'
wine.loc[wine['quality'].nsmallest(50).index, 'quality_category'] = 'Smallest'
wine.loc[wine['quality'].nlargest(50).index, 'quality_category'] = 'Largest'

# add the categories to this DataFrame
tsne_df['Category'] = wine['quality_category'].values

# create an interactive plot
fig = px.scatter(tsne_df, x='Dim1', y='Dim2', color='Category', hover_data=['color'])
fig.show()

#### While the tSNE does a great job at grouping by color, it is not a very good predictor of quality. 